## An example workflow for schism

This is a simple example for SCHISM that assumes that the grid and params files are given. Since no input options for meteo or bathymetry is given they are retrieved from an erddap server (GFS).

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.model as pm
from pyPoseidon.utils import data,point

In [ ]:
pyPoseidon.__version__

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs


hv.notebook_extension('bokeh')
#hv.notebook_extension('matplotlib')

### Define case

In [ ]:
#define in a dictionary the properties of the model..
dic={'solver':'schism',
     'grid_file': '/Users/brey/SCHISM/test_grid/hgrid.gr3', 
     'tag':'test',
     'start_date':'2017-2-1 0:0:0',
     'time_frame':'15H',
#     'end_date':'2013-2-5 0:0:0', #optional instead of time_frame
     'epath':'/Users/brey/SCHISM/v5.6.1/src/schism.ex_VL', #folder for solver executables
     'ncores': 4 , #number of cores
     'rpath':'./test/', #location of calc folder
     'conda_env': 'mpi3', # optional conda env for the solver
     'update':['meteo'] #update only meteo, keep dem
    }

## Initialize

In [ ]:
#initialize a model
b = pm.model(**dic)

### set it up

In [ ]:
b.set(**dic) #set it up 

In [ ]:
b.impl.meteo.impl.uvp.time

In [ ]:
b.impl.params.loc['rnday'] = .5

## Save to folder for execution 

In [ ]:
#set the run by saving the files
b.output()

In [ ]:
# save model info for further use
b.save()

In [ ]:
# save all matrices for further use
#b.pickle(path=path)

### execute

In [ ]:
#execute
b.run()

## Analysis of output 

In [ ]:
#%matplotlib inline

In [ ]:
otp = data(**dic)#,tag='eur025')# optional tag for data selection ,sa_date='2010-2-1 5:0:0',se_date='2010-2-1 22:0:0') # optionally set range for tide gauge data acquisition 

In [ ]:
otp.impl.Dataset

In [ ]:
otp.impl.Dataset.depth.max()

In [ ]:
otp.impl.Dataset.splot.contourf('depth' ,vmin=otp.impl.Dataset.depth.min(), vmax=otp.impl.Dataset.depth.max())

In [ ]:
otp.impl.Dataset.splot.grid()

In [ ]:
otp.impl.Dataset.splot.quiver('dahv',it=10,scale=.05)

### Visualize

In [ ]:
otp.impl.Dataset.splot.frames('elev',title='SSH')#,vmin=-.5, vmax=.5, nv=20)

In [ ]:
otp.impl.Dataset.splot.qframes('wind_speed',title='Vel',scale=30.)

## Observation points / Validation

In [ ]:
otp.impl.obs.locations

In [ ]:
# Get with index number
p = otp.impl.obs.iloc(2)
print p.head()
print p.tail()

In [ ]:
#get with Station Name  
p = otp.impl.obs.loc('Denmark - Hirtshals')
p.head()

In [ ]:
#plot
ax = p.plot()

### sample nearest point from simulation 

In [ ]:
 plat,plon = otp.impl.obs.locations.loc[2,['lat','lon']]

In [ ]:
ts = point(lon=plon,lat=plat,data=otp.impl.Dataset)
ts.stseries(var='elev')

In [ ]:
ts.elev

In [ ]:
ts.elev.plot()

In [ ]:
## Join the graphs
ax = p.plot(figsize=(14,10))
ts.elev.plot(ax=ax,color=['r'])
ax.legend(list(p.columns.values) + ['Schism'], loc='center left',bbox_to_anchor=(1.1, 0.5))